In [1]:
import tensorflow as tf, cv2, albumentations as A, os, numpy as np
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate, Dropout, BatchNormalization, Conv2DTranspose
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.utils import image_dataset_from_directory, plot_model
from sklearn.model_selection import train_test_split

In [2]:
tf.config.list_physical_devices('GPU')

I0000 00:00:1735355653.180022     529 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1735355653.395958     529 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1735355653.396042     529 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [3]:
os.chdir("/mnt/e/UCSD/Senior_Year/Senior_Design")

### Setting up model

In [4]:
# Define the augmentation pipeline
augmentation_pipeline = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),
    A.RandomRotate90(p=0.5),
    A.ShiftScaleRotate(shift_limit=0.05, scale_limit=0.05, rotate_limit=15, p=0.5),
    A.RandomBrightnessContrast(p=0.2)
])

# Function to apply augmentations dynamically
def augment_on_the_fly(image, mask):
    augmented = augmentation_pipeline(image=image, mask=mask)
    return augmented['image'], augmented['mask']

In [5]:
# Data generator for on-the-fly augmentation
def data_generator(image_paths, mask_paths, batch_size):
    while True:
        for i in range(0, len(image_paths), batch_size):
            batch_images = []
            batch_masks = []
            for j in range(i, min(i + batch_size, len(image_paths))):
                # Load image and mask
                image = cv2.imread(image_paths[j], cv2.COLOR_BGR2RGB)
                mask = cv2.imread(mask_paths[j], cv2.IMREAD_GRAYSCALE)
                
                # Resize to match input shape
                image = cv2.resize(image, (512, 512))
                mask = cv2.resize(mask, (512, 512))

                # Apply augmentation
                aug_image, aug_mask = augment_on_the_fly(image, mask)

                # Normalize
                aug_image = aug_image / 255.0
                aug_mask = aug_mask[..., np.newaxis] / 255.0  # Add channel dimension

                batch_images.append(aug_image)
                batch_masks.append(aug_mask)

            yield np.array(batch_images, dtype=np.float32), np.array(batch_masks, dtype=np.float32)

In [6]:
# # Define a convolutional block
# def conv_block(inputs, filters, kernel_size=(3, 3), activation='relu', padding='same'):
#     x = Conv2D(filters, kernel_size, activation=activation, padding=padding)(inputs)
#     x = BatchNormalization()(x)
#     x = Conv2D(filters, kernel_size, activation=activation, padding=padding)(x)
#     x = BatchNormalization()(x)
#     return x

# # Define an encoder block
# def encoder_block(inputs, filters):
#     x = conv_block(inputs, filters)
#     p = MaxPooling2D(pool_size=(2, 2))(x)
#     return x, p

# # Define a decoder block
# def decoder_block(inputs, skip_features, filters):
#     x = Conv2DTranspose(filters, kernel_size=(2, 2), strides=(2, 2), padding='same')(inputs)
#     x = concatenate([x, skip_features])
#     x = conv_block(x, filters)
#     return x

# # Define the Multi-Scale U-Net model
# def multi_scale_unet(input_shape=(512, 512, 3), num_classes=1):
#     inputs = Input(input_shape)

#     # Encoder path
#     s1, p1 = encoder_block(inputs, 64)
#     s2, p2 = encoder_block(p1, 128)
#     s3, p3 = encoder_block(p2, 256)
#     s4, p4 = encoder_block(p3, 512)

#     # Bridge
#     b1 = conv_block(p4, 1024)

#     # Decoder path
#     d1 = decoder_block(b1, s4, 512)
#     d2 = decoder_block(d1, s3, 256)
#     d3 = decoder_block(d2, s2, 128)
#     d4 = decoder_block(d3, s1, 64)

#     # Output layer
#     outputs = Conv2D(num_classes, (1, 1), activation='sigmoid' if num_classes == 1 else 'softmax')(d4)

#     # Model
#     model = Model(inputs, outputs, name="Multi-Scale-U-Net")
#     return model

# # Instantiate the model
# input_shape = (512, 512, 3)
# num_classes = 1  # Use 1 for binary segmentation, or adjust for multi-class segmentation
# model = multi_scale_unet(input_shape, num_classes)

# # Compile the model
# model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
#               loss='binary_crossentropy' if num_classes == 1 else 'categorical_crossentropy',
#               metrics=['accuracy'])

# # Display the model summary
# model.summary()

In [9]:
### Lighter Model
# Define a lighter convolutional block
def conv_block(inputs, filters, kernel_size=(3, 3), activation='relu', padding='same'):
    x = Conv2D(filters, kernel_size, activation=activation, padding=padding)(inputs)
    x = BatchNormalization()(x)
    return x

# Define an encoder block
def encoder_block(inputs, filters):
    x = conv_block(inputs, filters)
    p = MaxPooling2D(pool_size=(2, 2))(x)
    return x, p

# Define a decoder block
def decoder_block(inputs, skip_features, filters):
    x = Conv2DTranspose(filters, kernel_size=(2, 2), strides=(2, 2), padding='same')(inputs)
    x = concatenate([x, skip_features])
    x = conv_block(x, filters)
    return x

# Define a lighter Multi-Scale U-Net model
def multi_scale_unet(input_shape=(512, 512, 3), num_classes=1):
    inputs = Input(input_shape)

    # Encoder path
    s1, p1 = encoder_block(inputs, 32)
    s2, p2 = encoder_block(p1, 64)
    s3, p3 = encoder_block(p2, 128)
    s4, p4 = encoder_block(p3, 256)

    # Bridge
    b1 = conv_block(p4, 512)

    # Decoder path
    d1 = decoder_block(b1, s4, 256)
    d2 = decoder_block(d1, s3, 128)
    d3 = decoder_block(d2, s2, 64)
    d4 = decoder_block(d3, s1, 32)

    # Output layer
    outputs = Conv2D(num_classes, (1, 1), activation='sigmoid' if num_classes == 1 else 'softmax')(d4)

    # Model
    model = Model(inputs, outputs, name="Lighter-Multi-Scale-U-Net")
    return model

# Instantiate the model
input_shape = (512, 512, 3)
num_classes = 1  # Use 1 for binary segmentation, or adjust for multi-class segmentation
model = multi_scale_unet(input_shape, num_classes)

# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
              loss='binary_crossentropy' if num_classes == 1 else 'categorical_crossentropy',
              metrics=['accuracy'])

# Display the model summary
model.summary()

I0000 00:00:1735185904.674865    6793 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1735185904.675052    6793 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1735185904.675094    6793 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1735185905.858780    6793 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1735185905.858884    6793 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-12-25

Model: "Lighter-Multi-Scale-U-Net"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 512, 512,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 512, 512,  │        896 │ input_layer[0][0] │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 512, 512,  │        128 │ conv2d[0][0]      │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d       │ (None, 256, 256,  │          0 │ batch_normalizat… │
│ (MaxPooling2D)      │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 256, 256,  │     18,496 │ max_pooling2d[0]… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 256, 256,  │        256 │ conv2d_1[0][0]    │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_1     │ (None, 128, 128,  │          0 │ batch_normalizat… │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_2 (Conv2D)   │ (None, 128, 128,  │     73,856 │ max_pooling2d_1[… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 128, 128,  │        512 │ conv2d_2[0][0]    │
│ (BatchNormalizatio… │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_2     │ (None, 64, 64,    │          0 │ batch_normalizat… │
│ (MaxPooling2D)      │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_3 (Conv2D)   │ (None, 64, 64,    │    295,168 │ max_pooling2d_2[… │
│                     │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 64, 64,    │      1,024 │ conv2d_3[0][0]    │
│ (BatchNormalizatio… │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_3     │ (None, 32, 32,    │          0 │ batch_normalizat… │
│ (MaxPooling2D)      │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_4 (Conv2D)   │ (None, 32, 32,    │  1,180,160 │ max_pooling2d_3[… │
│                     │ 512)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 32, 32,    │      2,048 │ conv2d_4[0][0]    │
│ (BatchNormalizatio… │ 512)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_transpose    │ (None, 64, 64,    │    524,544 │ batch_normalizat… │
│ (Conv2DTranspose)   │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 64, 64,    │          0 │ conv2d_transpose

 Total params: 3,838,497 (14.64 MB)

 Trainable params: 3,835,553 (14.63 MB)

 Non-trainable params: 2,944 (11.50 KB)

### Loading images

In [10]:
image_prefix, mask_prefix = "image_dataset_1/original/", "image_dataset_1/mask/"
ip, mp = [image_prefix + n for n in os.listdir(image_prefix)], [mask_prefix + n for n in os.listdir(mask_prefix)]

In [ ]:
batch_size = 5
train_gen = data_generator(ip, mp, batch_size)
steps_per_epoch = len(ip) // batch_size

model.fit(train_gen, steps_per_epoch=steps_per_epoch, epochs=20, verbose = 2)

In [ ]:
model = load_model()